# Manipulation des données de nuscenes avec panda

In [ ]:
%matplotlib inline

from nuscenes.nuscenes import NuScenes
from nuscenes.can_bus.can_bus_api import NuScenesCanBus

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# data/sets/nuscenes
# D:\Utilisateurs\Alexandre\Repertoire_D\nuscenes\v1 0-trainval01

nusc = NuScenes(version='v1.0-mini', dataroot='data/sets/nuscenes')
nusc_can = NuScenesCanBus(dataroot='data/sets/nuscenes')

#nusc2 = NuScenes(version='v1.0-trainval', dataroot='D:/Utilisateurs/Alexandre/Repertoire_D/nuscenes/v1.0-trainval01')

In [ ]:
nusc.list_scenes()

In [ ]:
scene_name = 'scene-0061'
dic_scene = nusc_can.get_messages(scene_name,'vehicle_monitor')
#print(test)
features = ["vehicle_speed","steering","throttle","left_signal","right_signal"]
df_scene = pd.DataFrame.from_dict(dic_scene)[features]

print(df_scene)

In [ ]:
scene_name = 'scene-1100'
my_scene_token = nusc.field2token('scene', 'name', scene_name)[0]
nusc.render_scene_channel(my_scene_token, 'CAM_FRONT')

## Premier test: essayer d'apprendre quand mettre le clignotant 
Note: juste un essai pour prendre en main le dataset, avec seulement la vitesse et l'inclinaison du volant je ne pense pas qu'il soit possible de prévoir quand mettre un clignotant, il manque certaines informations (trajectoire notamment)

### 1/ Prétraitement des données

In [ ]:
all_scene = [ s["name"] for s in nusc.scene]
print(all_scene)

In [ ]:
# Mets des 1 tout le temps pour le clignotant au lieu d'une alternance par défaut
def fill_signal(df,signal):
    i = 0
    index = df.columns.get_loc(signal)
    while df[signal][i] != 1:
        i += 1
    while i  < len(df) and sum(df[signal][i:i+4]) >= 1:
        df.iat[i,index] = 1
        i += 1
    return df

In [ ]:
#Ajoute une nouvelle colonne où les valeurs sont: 0(rien), 1(clignotant gauche), 2(clignotant droit)
#Plus pratique "fill_signal" car on aura une seul colonne Y pour l'apprentissage
def add_signal_column(df,signal):
    i = 0
    tab = []
    while df[signal][i] != 1:
        tab += [0]
        i += 1
    while i  < len(df) and sum(df[signal][i:i+4]) >= 1:
        if signal == "right_signal":
            tab += [2]
        else:
            tab += [1]
        i += 1
    while i < len(df):
        tab += [0]
        i += 1
    df["signal"] = tab
    return df

In [ ]:
blackint = nusc_can.can_blacklist
blacklist = [ "scene-0"+ str(i) for i in blackint]
print(blacklist)
print( "%s" in all_scene)

In [ ]:
tab = []
for s in all_scene[:300]:
    #print(s)
    if s not in blacklist:
        dic_scene = nusc_can.get_messages(s,'vehicle_monitor')
        features = ["vehicle_speed","steering","throttle","left_signal","right_signal"]
        df_scene = pd.DataFrame.from_dict(dic_scene)[features]
        if df_scene["left_signal"].any():
            #df_scene = fill_signal(df_scene,"left_signal")
            new_df = add_signal_column(df_scene,"left_signal")
            #print(new_df)
        if df_scene["right_signal"].any():
            #df_scene = fill_signal(df_scene,"right_signal")
            new_df = add_signal_column(df_scene,"right_signal")
            #print(new_df)
        tab += [new_df]
    
df_total = pd.concat(tab)
print(df_total)

### 2/ Apprentissage

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import svm, neighbors
import random

In [ ]:
features = ["vehicle_speed","steering","throttle"]
X = df_total[features]
y = df_total["signal"]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 1)

#### 2/1/ K plus proches voisins

In [ ]:
model = neighbors.KNeighborsClassifier()
model.fit(X_train,y_train)
model.score(X_test,y_test)


0.80 avec les 1000 scènes (can bus), c'est pas mal pour un modèle basique, on peut améliorer ça

## Deuxième test: anticiper la vitesse avec un véhicule en face

Principe de base (idée): 
    - le système de pilotage envoie les postions gps à suivre, la vitesse et l'angle recommandés à l'algorithme de gestion
    - l'algorithme de gestion en fonction de la situation va envoyer sur un modèle pour donner une nouvelle valeur de la vitesse et de l'angle par rapport à l'environnement
    - plusieurs modèles, mais en premier un basique : s'il y a personne en face on change rien, sinon calcul de la nouvelle vitesse par rapport au véhicule devant.

Pour cela, il nous faut plusieurs données pour ce modèle:
    - position, vitesse, orientation du véhicule égo (nous) et du véhicule en face
    - à compléter?



### 1/ Recherche de ces données:

In [ ]:
scene_test = nusc.scene[0]
scene_test

In [ ]:
sample = nusc.get('sample',scene_test['first_sample_token'])
ann = sample['anns'][18]
ann_meta = nusc.get('sample_annotation', ann)
ann_meta